In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # to read records completely
import pickle
import numpy as np
import itertools
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

## Previous data prep necessary for c) and d)

In [3]:
# Importing the data scraped and pre-processed in part A
df = pd.read_pickle('data_clean.pkl')

In [4]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True) # clean some punctuation errors (nissan.)
models = models[~models.brand.isin(['car', 'sedan', 'problem'])] # remove rows that are not brands

In [5]:
df.Message.str.contains('said').sum()

370

In [6]:
#.lower() on all words
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

### Replace models with brands

In [7]:
messages = df.Message_words.to_list()

In [8]:
# numpy.where to do the replacement. Replaces only first appearance of brand.
# ~ 5-10 minutes of execute
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [9]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1 # if we want to count all words

In [10]:
print(f'{c} changes of models')

6368 changes of models


In [11]:
df[['Message_words', 'Message_words_v2']].head(10)

,Message_words,Message_words_v2
0,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]"
1,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]"
2,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]"
3,"[looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packageinfiniti, q45, premium, pckgaudi, a6, 42, loaded, audi, a8, stripped, loadedchicago, wintershad, blizzaks, gsworked, fine, 95q, pig, snow, im, sure, 2002q, world, away, ive, heard, audi, qc, problems, like, maintenance, included, im, happy, lexus, driver, q, appears, offer, lease, economy, would, think, could, command, attractive, package, go, wrong, 4, missing, something]","[looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packageinfiniti, q45, premium, pckgaudi, audi, 42, loaded, audi, audi, stripped, loadedchicago, wintershad, blizzaks, gsworked, fine, 95q, pig, snow, im, sure, 2002q, world, away, ive, heard, audi, qc, problems, like, maintenance, included, im, happy, lexus, driver, q, appears, offer, lease, economy, would, think, could, command, attractive, package, go, wrong, 4, missing, something]"
4,"[generally, 34, year, old, car, range, good, one, basically, sound, possible, get, one, still, warrenty, make, sure, trusted, maxima, mechanic, goes, thoroughly, v6, one, best, rated, 10, years, personally, amazed, great, strides, american, quality, recently, olds, may, bad, idea]","[generally, 34, year, old, car, range, good, one, basically, sound, possible, get, one, still, warrenty, make, sure, trusted, nissan, mechanic, goes, thoroughly, v6, one, best, rated, 10, years, personally, amazed, great, strides, american, quality, recently, olds, may, bad, idea]"
5,"[maxima, intrigue, excellent, choices, midsize, sedans]","[nissan, intrigue, excellent, choices, midsize, sedans]"
6,"[kind, budget, would, go, a6, 42, best, combination, sport, luxury, amongst, 4, choices]","[kind, budget, would, go, audi, 42, best, combination, sport, luxury, amongst, 4, choices]"
7,"[actually, agree, a6, looked, specs, edmunds, a6, a8, 4000, lbs, considering, a8, gets, bump, hp, a6, dont, think, performance, difference, issue, considering, fact, a6, would, possibly, small, based, rest, list, say, go, a8, think, youll, find, awd, best, addition, car, could, ever, hoped]","[actually, agree, aud

In [12]:
#Now we have cases where the text says "nissan", "nissan" because we replaced the model. Let's clean that
#Removing duplicates in lists while keeping order
def remove_consecutive_duplicate (text): #removes consecutive duplicates
    return np.array([i for i, j in itertools.groupby(text)])

df["Message_words_v3"] = df["Message_words_v2"].apply(remove_consecutive_duplicate)

## Link attributes to models

In [13]:
# first lets count brand mentions in each record
#Get a unique list of brands
brands_list = models.brand.drop_duplicates().to_list()

brands_list.extend(['lexus', 'ferrari', 'merzedesbenz', 'tesla','gm', 'peugeot', 'jeep', 'bentley', 'fiat'])

In [14]:
messages = df.Message_words_v3.to_list()

#Get a nested list of brands mentioned in reviews
brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

C:\Users\louis\AppData\Local\Temp/ipykernel_15520/1794413302.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if len(np.where(m == brand)[0]) >0:


In [15]:
df['Brands_in_message'] = brands_in_message
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))
df['Qty_brands'].value_counts()

# 1291 messages don't mention any brand (remove them from analysis?)
# Most messages discuss a single brand
# Also common to compare 2 brands

0     1244
1     1175
2      774
3      464
4      235
5      120
6       74
7       31
8       17
10       7
11       3
12       3
9        2
16       1
13       1
Name: Qty_brands, dtype: int64

In [16]:
df.loc[df.Qty_brands == 0, 'Message_words_v3'].head(10) # to review records with no brand mentions
# most of them are for seeking advice

0                                                                                       [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]
2     [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]
8                                                                                                                                                                                                                                                                                      [id, pass, intrigue, one, reaso

We can assign attributes mentioned in the reviews, like the power of a car, to a specific brand. For example, in the sentence "I like the BMW for its power. On the other hand, the Honda is reliable" we would want to assign the the attribute "power" to "BMW" and "reliable" to "Honda". We have created functions for two different approaches:
1) assign all words found between 1st brand mention and next brand mention to 1st brand

2) assign n words to each side of the brand mention to the brand. We found n = 4 to work best. 

In [17]:
# First approach: link all words found between brand mention and next brand mention.
# Except for first brand which also receives words since beginning of message

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_in_the_right(m, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)
        
        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    return review



get_attr_in_the_right(m, brands_list) #was brand_list, ASK CARLOS    

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [18]:
# now we run it for all records

brand_reviews_1st_approach = []

for m in df.Message_words_v3:

    brand_reviews_1st_approach.append(get_attr_in_the_right(m, brands_list))

# generates a list (1 entry per row) of dictionaries {brand_1: part of text corresponding, 
# brand_2: part of text corresponding,}

C:\Users\louis\AppData\Local\Temp/ipykernel_15520/2213854526.py:10: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [19]:
# 2nd approach: n words to each side 

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_n_per_side(m, n, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))
    
    #print(len(brand_dic))

    brand_list = list(brand_dic.values()) + [len(m)]
    
    #print(brand_list)

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)

        left_loc = (brand_list[i] - n)
        left_loc = max(0, left_loc) # to avoid out of range indexing
        
        right_loc = (brand_list[i] + n)
        right_loc = min(len(m), right_loc) # to avoid OOF indexing
        
        review[tup] = m[left_loc:right_loc]
            
    return review



get_attr_n_per_side(m, 5, brands_list)        

{'honda': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new'],
       dtype='<U12'),
 'toyota': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new',
        'nissan'], dtype='<U12'),
 'nissan': array(['honda', 'toyota', 'choices', 'sedan', 'new', 'nissan', 'looks',
        'impressive', 'would', 'wait'], dtype='<U12'),
 'bmw': array(['wait', 'reliability', '2002', 'proventhe', 'new', 'bmw', '525',
        'another', 'nice', 'car'], dtype='<U12')}

In [20]:
m = df.Message_words_v3[9] 

m

array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new',
       'nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
       '2002', 'proventhe', 'new', 'bmw', '525', 'another', 'nice', 'car',
       'rated', 'highly', 'many', 'auto', 'publications'], dtype='<U12')

In [21]:
get_attr_n_per_side(m, 4, brands_list)

{'honda': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan'],
       dtype='<U12'),
 'toyota': array(['would', 'recommend', 'honda', 'toyota', 'choices', 'sedan', 'new'],
       dtype='<U12'),
 'nissan': array(['toyota', 'choices', 'sedan', 'new', 'nissan', 'looks',
        'impressive', 'would'], dtype='<U12'),
 'bmw': array(['reliability', '2002', 'proventhe', 'new', 'bmw', '525', 'another',
        'nice'], dtype='<U12')}

In [22]:
get_attr_in_the_right(m, brands_list)        

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [23]:
# now we run it for all records
brand_reviews_2nd_approach = []

for m in df.Message_words_v3:

    brand_reviews_2nd_approach.append(get_attr_n_per_side(m,4, brands_list))

# generates a list (1 entry per row) of dictionaries {brand_1: part of text corresponding, 
# brand_2: part of text corresponding,}

C:\Users\louis\AppData\Local\Temp/ipykernel_15520/25208796.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [239]:
len(brand_reviews_2nd_approach) # each record becomes a single dictionary with all its brand mentions and linked words

4151

In [25]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
# creates dictionary: each brand mentioned is a key, value is all words assigned to that brand 
# according to 1st method get_attr_in_the_right()

review_accum_1st = {i:[] for i in brands_list} #deprecated dictionary but still interesting

for brand in brands_list:
    for review in brand_reviews_1st_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_1st[brand].extend(review[single_review])

In [26]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
# creates dictionary: each brand mentioned is a key, value is all words assigned to that brand
# according to 2nd method get_attr_n_per_side()

review_accum_2nd = {i:[] for i in brands_list} #deprecated dictionary but still interesting

for brand in brands_list:
    for review in brand_reviews_2nd_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_2nd[brand].extend(review[single_review])

In [31]:
## from here, we are ready to start counting attributes per brand
## we also need to consider replacing some similar attributes before doing the final count 


In [35]:
df.head(10)

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words,Message_words_v2,Message_words_v3,Brands_in_message,Qty_brands
0,2001-09-01,pat,"Need help choosing your next vehicle? Tell us your price range, are you thinking New or Used, buying or leasing, what features are must-haves, what other thoughts are on your mind, and let us give you a hand! Tell us your criteria as specifically as you can.You'll find lots of helpful folks here who can give you useful suggestions.","10,421",Member,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]",[],0
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year and my first choice is a Nissan Maxima. I've read that they are very reliable and hold their value well. My budget is probably going to allow for a 98-99 and I was wondering if it's worth buying a car that's almost 3-4 years old. Any suggestions or comments would help a lot.willow25",2,Member,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]",[nissan],1
2,2001-10-01,dindak,"Buy a 2001/ 2002 Oldsmobile Intrigue. It's one of the best sedans on the market with great handling and one of the best V6 engines around, a 3.5L DOHC producing 215HP. The deals should be great this year as it's the last on for Intrigue and the warrany on them has been extended to 5 years. The car is recommended by most magazines (including Consumer Reports), but is unfortunately over looked by most consumers. It is the sleeper of all sleepers on the market in my opinion.","6,632",Member,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]",[],0
3,2001-10-01,peteri1,"Am looking at 1 of 4 choice

In [40]:
# Create Series with all words 
words = pd.Series([i for review in df.Message_words_v3 for i in review])

In [41]:
# Get value count for words excluding brands
words[~words.isin(brands_list)].value_counts().head(60)


car            3874
like           1900
would          1900
get            1797
new            1632
cars           1554
good           1491
one            1447
im             1120
also           1115
dont           1055
drive          1013
miles           979
much            959
used            945
want            932
years           928
price           917
better          916
think           902
well            845
looking         833
really          790
buy             740
know            722
go              671
need            654
year            651
even            650
could           628
great           624
ive             590
vehicle         576
time            570
might           569
driving         563
lot             545
look            541
something       533
best            528
going           525
id              514
old             513
engine          508
3               504
nice            498
civic           496
still           489
probably        489
reliability     485


In [42]:
words_count = words.value_counts()
words_nobrands_count = words[~words.isin(brands_list)].value_counts()

### Stemming

In [43]:
#Stemming
stemmer = PorterStemmer()

In [44]:
def stem_list (text):
    return pd.Series([stemmer.stem(w) for w in text])

In [45]:
stemmed_words = stem_list(words)
len(stemmed_words)

227214

In [46]:
stemmed_words.value_counts().head(60)

car           5429
get           2274
like          2221
would         1900
look          1821
honda         1746
drive         1707
new           1632
year          1579
one           1556
good          1493
toyota        1341
use           1283
go            1196
price         1154
want          1150
im            1120
also          1115
think         1076
buy           1075
mile          1074
dont          1055
nissan         963
much           959
better         919
reliabl        903
vehicl         892
need           889
well           845
know           793
realli         790
volkswagen     742
make           737
model          730
time           704
lot            692
even           653
engin          630
could          628
great          624
dealer         605
ive            590
seat           586
might          569
chevrolet      559
sedan          549
consid         547
old            536
someth         533
leas           533
civic          529
best           529
find        

In [47]:
stemmed_words[~stemmed_words.isin(brands_list)].value_counts().head(60)


car        5429
get        2274
like       2221
would      1900
look       1821
drive      1707
new        1632
year       1579
one        1556
good       1493
use        1283
go         1196
price      1154
want       1150
im         1120
also       1115
think      1076
buy        1075
mile       1074
dont       1055
much        959
better      919
reliabl     903
vehicl      892
need        889
well        845
know        793
realli      790
make        737
model       730
time        704
lot         692
even        653
engin       630
could       628
great       624
dealer      605
ive         590
might       569
sedan       549
consid      547
old         536
someth      533
leas        533
civic       529
best        529
find        526
nice        519
id          515
3           504
cost        500
probabl     490
still       489
seem        482
less        479
test        479
option      473
say         473
around      471
thing       470
dtype: int64

In [48]:
stemwords_count = stemmed_words.value_counts()
stemwords_nobrands_count = stemmed_words[~stemmed_words.isin(brands_list)].value_counts()

In [49]:
#For words_nobrands_count remove words that appear 2x or less
words_nobrands_count = words_nobrands_count[words_nobrands_count > 2]
words_nobrands_count

car         3874
like        1900
would       1900
get         1797
new         1632
            ... 
happiest       3
switched       3
caryou         3
pushes         3
talks          3
Length: 5975, dtype: int64

In [50]:
len(words_nobrands_count)

5975

In [ ]:
# This csv is used to create our attribute map
#words_nobrands_count.to_csv("attribute_count_final.csv")

### Importing attribute map

Attribute mapping was done by manually browsing rows of xlsx and linking words we found relevant to a common key

In [51]:
df_map = pd.read_excel("attribute_count_final.xlsx", sheet_name = 0)

In [52]:
df_map = df_map.drop('Unnamed: 4', axis=1)

In [53]:
df_map

,word,freq,attribute_flg,attribute_synonym
0,price,737.0,1.0,affordability
1,money,338.0,1.0,affordability
2,lease,311.0,1.0,affordability
3,value,273.0,1.0,affordability
4,cost,270.0,1.0,affordability
...,...,...,...,...
5356,nope,3.0,NaN,NaN
5357,scrape,3.0,NaN,NaN
5358,850.0,3.0,NaN,NaN
5359,modelsi,3.0,NaN,NaN


In [54]:
#Replace np.nan in df_map
df_map['attribute_flg'] = df_map['attribute_flg'].replace(np.nan, 0)
df_map.attribute_synonym.fillna(df_map.word, inplace=True)


In [55]:
df_pivot_map = df_map[df_map.attribute_flg == 1.0].groupby('attribute_synonym').freq.sum().sort_values(ascending=False)

In [56]:
df_pivot_map.head(10)

attribute_synonym
affordability     4509.0
sustainability    1902.0
size              1829.0
driveability      1535.0
engine            1433.0
visual            1202.0
reliable          1012.0
transmission       998.0
comfort            893.0
technological      867.0
Name: freq, dtype: float64

The top 5 attributes are: affordability, sustainability, size, driveability and engine

In [253]:
# Replace the attributes in df.Message_words_v3
def replace_attributes(m):
    for i,row in df_map.iterrows():
        m = np.where(m == row['word'], row['attribute_synonym'], m)
    return m

In [130]:
replace_attributes(df.Message_words_v3.loc[2])

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/764214923.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  m = np.where(m == row['word'], row['attribute_synonym'], m)


array(['buy', '2001', '2002', 'oldsmobile', 'intrigue', 'one', 'best',
       'sedans', 'market', 'great', 'driveability', 'one', 'best',
       'engine', 'engine', 'around', '35l', 'dohc', 'producing', '215hp',
       'deals', 'great', 'year', 'last', 'intrigue', 'warrany',
       'extended', '5', 'years', 'car', 'recommended', 'magazines',
       'including', 'consumer', 'reports', 'unfortunately', 'looked',
       'consumers', 'sleeper', 'sleepers', 'market', 'opinion'],
      dtype='<U158')

In [134]:
# Replaces attributes
# Takes ~20-30 mins to run
df["Message_words_v3"] = df["Message_words_v3"].apply(replace_attributes)

C:\Users\louis\AppData\Local\Temp/ipykernel_21972/764214923.py:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  m = np.where(m == row['word'], row['attribute_synonym'], m)


In [80]:
# Drop Message_words_v2 for size
df_saved = df.copy()
df = df.drop("Message_words_v2", axis=1)

In [82]:
#Create checkpoint csv to avoid long function call
#df.to_pickle("data_checkpoint.pkl")

# Calculating lift scores

In [57]:
#Import checkpoint csv to avoid long function call
df = pd.read_pickle("data_checkpoint.pkl")

In [81]:
df

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words,Message_words_v3,Brands_in_message,Qty_brands
0,2001-09-01,pat,"Need help choosing your next vehicle? Tell us your price range, are you thinking New or Used, buying or leasing, what features are must-haves, what other thoughts are on your mind, and let us give you a hand! Tell us your criteria as specifically as you can.You'll find lots of helpful folks here who can give you useful suggestions.","10,421",Member,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, affordability, sustainability, thinking, new, used, buying, affordability, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]",[],0
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year and my first choice is a Nissan Maxima. I've read that they are very reliable and hold their value well. My budget is probably going to allow for a 98-99 and I was wondering if it's worth buying a car that's almost 3-4 years old. Any suggestions or comments would help a lot.willow25",2,Member,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, ive, read, reliable, hold, affordability, well, affordability, probably, going, allow, 9899, wondering, affordability, buying, car, thats, almost, 34, years, age, suggestions, comments, would, help, lotwillow25]",[nissan],1
2,2001-10-01,dindak,"Buy a 2001/ 2002 Oldsmobile Intrigue. It's one of the best sedans on the market with great handling and one of the best V6 engines around, a 3.5L DOHC producing 215HP. The deals should be great this year as it's the last on for Intrigue and the warrany on them has been extended to 5 years. The car is recommended by most magazines (including Consumer Reports), but is unfortunately over looked by most consumers. It is the sleeper of all sleepers on the market in my opinion.","6,632",Member,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, driveability, one, best, engine, engine, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]",[],0
3,2001-10-01,peteri1,"Am looking at 1 of 4 choices to replace my 99GS400. I've read everything here and on other forums and have seen much conflicting info:4 options:Lexus LS430 with custom/luxury packageInfiniti Q45 with premium pckgAudi A-6 4.2 loaded Audi A-8 not stripped, but not loadedChicago winters--had Blizzaks on the GS--worked fine. My 95Q was pig in snow, but I'm sure 2002Q is a world away from that. I've heard Audi has some QC problems, but I like their maintenance being included. I'm a happy Lexus driver. The Q appears to offer the most for the $$. Will lease and in this economy, I would think I could command an attractive package. Can I go wrong with any of the 4? Am I missing something?",21,Member,"[looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packa

In [59]:
#Function to calculate lift between a brand and an attribute

def calculate_lift(brand_name, attribute_name):
    
    #Used to get counts in separate table
    string_list = []
    
    #Get total number of reviews
    review_count = len(df)
    string_list.append(f"There are {review_count} reviews")

    #Count number of times brand appears across reviews
    brand_count = 0
    for review in  df.Message_words_v3:
        if brand_name in review:
            brand_count += 1
    string_list.append(f"{brand_name} appears {brand_count} times")
    
    #Get number of times attribute appears across reviews
    attribute_count = 0
    for review in df.Message_words_v3:
            if attribute_name in review:
                attribute_count += 1
    string_list.append(f"{attribute_name} appears {attribute_count} times")

    #Count number of times attribute and brand appear together
    attr_brand_count = 0
    for review in df.Message_words_v3:
        if brand_name in review and attribute_name in review:
            attr_brand_count += 1
    string_list.append(f"{attribute_name} and {brand_name} appears together {attr_brand_count} times")
    
    #Calculate lift
    lift = review_count * (attr_brand_count/(brand_count * attribute_count))
    
    return lift, string_list

In [60]:
calculate_lift('nissan', 'affordability')

(1.2506627489389488,
 ['There are 4151 reviews',
  'nissan appears 525 times',
  'affordability appears 2042 times',
  'affordability and nissan appears together 323 times'])

In [62]:
# Define top brands and attributes as found in parts A and C/D
top_attributes = ['affordability', 'sustainability', 'size', 'driveability', 'engine']
top_brands = ['honda','toyota','nissan','volkswagen','chevrolet']

In [63]:
# Initialize dataframes to store lift and support values
df_lift = pd.DataFrame(index=top_brands, columns = top_attributes)
df_lift_counts = pd.DataFrame(index=top_brands, columns = top_attributes)

In [64]:
for brand in top_brands:
    for attribute in top_attributes:
        df_lift.loc[brand, attribute], df_lift_counts.loc[brand, attribute] = calculate_lift(brand, attribute)

In [65]:
df_lift

,affordability,sustainability,size,driveability,engine
honda,1.236407,1.475249,1.333386,1.285018,1.665103
toyota,1.23449,1.572307,1.41119,1.358744,1.416618
nissan,1.250663,1.334892,1.466724,1.266514,1.612803
volkswagen,1.148263,1.176009,1.306777,1.334974,1.465157
chevrolet,1.212448,1.668234,1.774826,1.719377,2.052918


In [66]:
df_lift_counts

,affordability,sustainability,size,driveability,engine
honda,"[There are 4151 reviews, honda appears 924 times, affordability appears 2042 times, affordability and honda appears together 562 times]","[There are 4151 reviews, honda appears 924 times, sustainability appears 1078 times, sustainability and honda appears together 354 times]","[There are 4151 reviews, honda appears 924 times, size appears 1159 times, size and honda appears together 344 times]","[There are 4151 reviews, honda appears 924 times, driveability appears 874 times, driveability and honda appears together 250 times]","[There are 4151 reviews, honda appears 924 times, engine appears 804 times, engine and honda appears together 298 times]"
toyota,"[There are 4151 reviews, toyota appears 769 times, affordability appears 2042 times, affordability and toyota appears together 467 times]","[There are 4151 reviews, toyota appears 769 times, sustainability appears 1078 times, sustainability and toyota appears together 314 times]","[There are 4151 reviews, toyota appears 769 times, size appears 1159 times, size and toyota appears together 303 times]","[There are 4151 reviews, toyota appears 769 times, driveability appears 874 times, driveability and toyota appears together 220 times]","[There are 4151 reviews, toyota appears 769 times, engine appears 804 times, engine and toyota appears together 211 times]"
nissan,"[There are 4151 reviews, nissan appears 525 times, affordability appears 2042 times, affordability and nissan appears together 323 times]","[There are 4151 reviews, nissan appears 525 times, sustainability appears 1078 times, sustainability and nissan appears together 182 times]","[There are 4151 reviews, nissan appears 525 times, size appears 1159 times, size and nissan appears together 215 times]","[There are 4151 reviews, nissan appears 525 times, driveability appears 874 times, driveability and nissan appears together 140 times]","[There are 4151 reviews, nissan appears 525 times, engine appears 804 times, engine and nissan appears together 164 times]"
volkswagen,"[There are 4151 reviews, volkswagen appears 370 times, affordability appears 2042 times, affordability and volkswagen appears together 209 times]","[There are 4151 reviews, volkswagen appears 370 times, sustainability appears 1078 times, sustainability and volkswagen appears together 113 times]","[There are 4151 reviews, volkswagen appears 370 times, size appears 1159 times, size and volkswagen appears together 135 times]","[There are 4151 reviews, volkswagen appears 370 times, driveability appears 874 times, driveability and volkswagen appears together 104 times]","[There are 4151 reviews, volkswagen appears 370 times, engine appears 804 times, engine and volkswagen appears together 105 times]"
chevrolet,"[There are 4151 reviews, chevrolet appears 337 times, affordability appears 2042 times, affordability and chevrolet appears together 201 times]","[There are 4151 reviews, chevrolet appears 337 times, sustainability appears 1078 times, sustainability and chevrolet appears together 146 times]","[There are 4151 reviews, chevrolet appears 337 times, size appears 1159 times, size and chevrolet appears together 167 times]","[There are 4151 reviews, chevrolet appears 337 times, driveability appears 874 times, driveability and chevrolet appears together 122 times]","[There are 4151 reviews, chevrolet appears 337 times, engine appears 804 times, engine and chevrolet appears together 134 times]"


## Calculating lift from brand-attribute assignment lists
brand_reviews_1st_approach and brand_reviews_2nd_approach


In [68]:
# Get split reviews with replaced attributes
# generates a list (1 entry per row) of dictionaries {brand_1: part of text corresponding, 
# brand_2: part of text corresponding}
brand_reviews_1st_mapped = []

for m in df.Message_words_v3:

    brand_reviews_1st_mapped.append(get_attr_in_the_right(m, brands_list))

brand_reviews_2nd_mapped = []

for m in df.Message_words_v3:

    brand_reviews_2nd_mapped.append(get_attr_n_per_side(m,4, brands_list))

In [69]:
# Calculate split from list of dictionaries brand_reviews_1st_mapped and brand_reviews_2nd_mapped

def calculate_lift_split(brand_name, attribute_name, review_list):
    
    #Used to get counts in separate table
    string_list = []
    
    #Get total number of reviews
    #We define reviews as a part of a forum post that talks about a specific brand
    #Each review is therefore a split of the entire post, the split being made 
    # in 2 different ways as seen above (get_attr_in_the_right, get_attr_n_per_side )
    
    review_count = 0
    for review in review_list: #review_list is a list of dicts, so review is a dict
        review_count += len(review)
    string_list.append(f"There are {review_count} reviews about specific brands")

    #Count number of times brand appears across split reviews
    brand_count = 0
    for review in review_list: #review_list is a list of dicts, so review is a dict
        for key in review.keys():
            if brand_name == key:
                brand_count += 1
    string_list.append(f"{brand_name} appears {brand_count} times")
    
    #Count number of times attribute appears across split reviews
    attribute_count = 0
    for review in review_list:
        for value in review.values():
            if attribute_name in value:
                attribute_count += 1
    string_list.append(f"{attribute_name} appears {attribute_count} times")

    #Count number of times attribute and brand appear together
    attr_brand_count = 0
    for review in brand_reviews_1st_mapped:
        for key, value in review.items():
            if brand_name == key and attribute_name in value:
                attr_brand_count += 1
    string_list.append(f"{attribute_name} and {brand_name} appears together {attr_brand_count} times")
   
    #Calculate lift
    lift = review_count * (attr_brand_count/(brand_count * attribute_count))
    return lift, string_list

In [70]:
# Initialize dataframes
df_lift_1st_mapped = pd.DataFrame(index=top_brands, columns = top_attributes)
df_lift_1st_counts = pd.DataFrame(index=top_brands, columns = top_attributes)
df_lift_2nd_mapped = pd.DataFrame(index=top_brands, columns = top_attributes)
df_lift_2nd_counts = pd.DataFrame(index=top_brands, columns = top_attributes)

In [71]:
#Getting lift for brand_reviews_1st_mapped
for brand in top_brands:
    for attribute in top_attributes:
        df_lift_1st_mapped.loc[brand, attribute], df_lift_1st_counts.loc[brand, attribute] = calculate_lift_split(brand, attribute, brand_reviews_1st_mapped)

In [72]:
df_lift_1st_mapped

,affordability,sustainability,size,driveability,engine
honda,1.048563,1.16192,0.892129,0.722367,1.348502
toyota,0.889115,1.292405,0.945005,0.877506,0.868021
nissan,1.045377,1.028332,1.074311,0.852235,1.186682
volkswagen,1.052401,0.91195,0.806152,0.872247,1.162627
chevrolet,0.973494,1.256114,1.400055,1.327665,1.518565


In [73]:
df_lift_1st_counts

,affordability,sustainability,size,driveability,engine
honda,"[There are 6638 reviews about specific brands, honda appears 924 times, affordability appears 2165 times, affordability and honda appears together 316 times]","[There are 6638 reviews about specific brands, honda appears 924 times, sustainability appears 1082 times, sustainability and honda appears together 175 times]","[There are 6638 reviews about specific brands, honda appears 924 times, size appears 1224 times, size and honda appears together 152 times]","[There are 6638 reviews about specific brands, honda appears 924 times, driveability appears 905 times, driveability and honda appears together 91 times]","[There are 6638 reviews about specific brands, honda appears 924 times, engine appears 895 times, engine and honda appears together 168 times]"
toyota,"[There are 6638 reviews about specific brands, toyota appears 769 times, affordability appears 2165 times, affordability and toyota appears together 223 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, sustainability appears 1082 times, sustainability and toyota appears together 162 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, size appears 1224 times, size and toyota appears together 134 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, driveability appears 905 times, driveability and toyota appears together 92 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, engine appears 895 times, engine and toyota appears together 90 times]"
nissan,"[There are 6638 reviews about specific brands, nissan appears 525 times, affordability appears 2165 times, affordability and nissan appears together 179 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, sustainability appears 1082 times, sustainability and nissan appears together 88 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, size appears 1224 times, size and nissan appears together 104 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, driveability appears 905 times, driveability and nissan appears together 61 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, engine appears 895 times, engine and nissan appears together 84 times]"
volkswagen,"[There are 6638 reviews about specific brands, volkswagen appears 370 times, affordability appears 2165 times, affordability and volkswagen appears together 127 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, sustainability appears 1082 times, sustainability and volkswagen appears together 55 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, size appears 1224 times, size and volkswagen appears together 55 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, driveability appears 905 times, driveability and volkswagen appears together 44 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, engine appears 895 times, engine and volkswagen appears together 58 times]"
chevrolet,"[There are 6638 reviews about specific brands, chevrolet appears 337 times, affordability appears 2165 times, affordability and chevrolet appears together 107 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, sustainability appears 1082 times, sustainability and chevrolet appears together 69 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, size appears 1224 times, size and chevrolet appears together 87 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, driveability appears 905 times, driveability and chevrolet appears together 61 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, engine appears 895 times, engine and chevrolet appear

In [74]:
#Getting lift for brand_reviews_2nd_mapped
for brand in top_brands:
    for attribute in top_attributes:
        df_lift_2nd_mapped.loc[brand, attribute], df_lift_2nd_counts.loc[brand, attribute] = calculate_lift_split(brand, attribute, brand_reviews_2nd_mapped)

In [75]:
df_lift_2nd_mapped

,affordability,sustainability,size,driveability,engine
honda,3.393331,4.930184,2.563299,2.781883,3.570737
toyota,2.87733,5.483852,2.715227,3.379332,2.298459
nissan,3.383022,4.363354,3.086752,3.28201,3.142249
volkswagen,3.405753,3.869528,2.316267,3.35908,3.078554
chevrolet,3.150395,5.329866,4.022694,5.112924,4.021052


In [76]:
df_lift_2nd_counts

,affordability,sustainability,size,driveability,engine
honda,"[There are 6638 reviews about specific brands, honda appears 924 times, affordability appears 669 times, affordability and honda appears together 316 times]","[There are 6638 reviews about specific brands, honda appears 924 times, sustainability appears 255 times, sustainability and honda appears together 175 times]","[There are 6638 reviews about specific brands, honda appears 924 times, size appears 426 times, size and honda appears together 152 times]","[There are 6638 reviews about specific brands, honda appears 924 times, driveability appears 235 times, driveability and honda appears together 91 times]","[There are 6638 reviews about specific brands, honda appears 924 times, engine appears 338 times, engine and honda appears together 168 times]"
toyota,"[There are 6638 reviews about specific brands, toyota appears 769 times, affordability appears 669 times, affordability and toyota appears together 223 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, sustainability appears 255 times, sustainability and toyota appears together 162 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, size appears 426 times, size and toyota appears together 134 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, driveability appears 235 times, driveability and toyota appears together 92 times]","[There are 6638 reviews about specific brands, toyota appears 769 times, engine appears 338 times, engine and toyota appears together 90 times]"
nissan,"[There are 6638 reviews about specific brands, nissan appears 525 times, affordability appears 669 times, affordability and nissan appears together 179 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, sustainability appears 255 times, sustainability and nissan appears together 88 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, size appears 426 times, size and nissan appears together 104 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, driveability appears 235 times, driveability and nissan appears together 61 times]","[There are 6638 reviews about specific brands, nissan appears 525 times, engine appears 338 times, engine and nissan appears together 84 times]"
volkswagen,"[There are 6638 reviews about specific brands, volkswagen appears 370 times, affordability appears 669 times, affordability and volkswagen appears together 127 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, sustainability appears 255 times, sustainability and volkswagen appears together 55 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, size appears 426 times, size and volkswagen appears together 55 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, driveability appears 235 times, driveability and volkswagen appears together 44 times]","[There are 6638 reviews about specific brands, volkswagen appears 370 times, engine appears 338 times, engine and volkswagen appears together 58 times]"
chevrolet,"[There are 6638 reviews about specific brands, chevrolet appears 337 times, affordability appears 669 times, affordability and chevrolet appears together 107 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, sustainability appears 255 times, sustainability and chevrolet appears together 69 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, size appears 426 times, size and chevrolet appears together 87 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, driveability appears 235 times, driveability and chevrolet appears together 61 times]","[There are 6638 reviews about specific brands, chevrolet appears 337 times, engine appears 338 times, engine and chevrolet appears together 69 t